In [1]:
API_KEY = "6d17201f8d8043d0bb98961e8cbb3966"

In [2]:
import pandas as pd
import requests
import time

In [3]:
df = pd.read_csv('vgsales.csv')
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [5]:
print(df.isna().sum())
print('-'*10)
print(df.duplicated().sum())

Rank              0
Name              0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64
----------
0


In [6]:
missing_years = df[df['Year'].isna()].index

In [8]:
def get_game_release_year(game_name, api_key=API_KEY):
    url = 'https://api.rawg.io/api/games'
    params = {
        'key': api_key,
        'search': game_name,
        'page_size': 1,  
        'search_exact': True, 
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"API error: Status {response.status_code}")
            return None
        
        data = response.json()
        print(f"Api response for {game_name}: {data}") 
        if data.get('results') and len(data['results']) > 0:
            game = data['results'][0]
            release_date = game.get('released')
            if release_date:
                year = int(release_date.split('-')[0])
                print(f"Year found: {year}")
                return year
            else:
                print(f"release date not available {game_name}")
        else:
            print(f"No results for {game_name}")
    except Exception as e:
        print(f"error on {game_name}: {e}")
    return None

In [11]:
for idx in missing_years:
    game_name = df.loc[idx, 'Name']
    platform = df.loc[idx, 'Platform']
    print(f"\nserching: {game_name} ({platform})")
    
    year = get_game_release_year(game_name, API_KEY)
    if year:
        df.loc[idx, 'Year'] = year
    # else:
    #     print(f"Year not found {game_name}")
    
    time.sleep(1)  


serching: Madden NFL 2004 (PS2)
Api response for Madden NFL 2004: {'count': 1, 'next': None, 'previous': None, 'results': [{'slug': 'madden-nfl-2004', 'name': 'Madden NFL 2004', 'playtime': 0, 'platforms': [{'platform': {'id': 4, 'name': 'PC', 'slug': 'pc'}}, {'platform': {'id': 80, 'name': 'Xbox', 'slug': 'xbox-old'}}, {'platform': {'id': 15, 'name': 'PlayStation 2', 'slug': 'playstation2'}}, {'platform': {'id': 27, 'name': 'PlayStation', 'slug': 'playstation1'}}, {'platform': {'id': 105, 'name': 'GameCube', 'slug': 'gamecube'}}, {'platform': {'id': 24, 'name': 'Game Boy Advance', 'slug': 'game-boy-advance'}}], 'stores': None, 'released': '2003-08-12', 'tba': False, 'background_image': 'https://media.rawg.io/media/screenshots/543/5438fbc52f8aaee11f8a0b4eb3e7e255.jpg', 'rating': 3.83, 'rating_top': 4, 'ratings': [{'id': 4, 'title': 'recommended', 'count': 3, 'percent': 50.0}, {'id': 5, 'title': 'exceptional', 'count': 2, 'percent': 33.33}, {'id': 1, 'title': 'skip', 'count': 1, 'perce

In [12]:
print(df.isna().sum())

Rank             0
Name             0
Platform         0
Year            52
Genre            0
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
Global_Sales     0
dtype: int64


In [15]:
df_missing_year = df[df['Year'].isnull()].copy()

In [18]:
df.fillna({'Publisher':'Unknown'},inplace=True) 
df.dropna(axis=0,inplace=True)

In [19]:
print(df.isna().sum())

Rank            0
Name            0
Platform        0
Year            0
Genre           0
Publisher       0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
dtype: int64


In [20]:
df['Year'] = df['Year'].astype('int')

In [22]:
selection = df.loc[df['Year']>2016].index
df = df.drop(selection)

In [23]:
df.to_csv('vgsales_clean.csv',index=False)

In [ ]:
df[df['Name']==]